# MintPy

## configure file

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from mintpy.objects import ifgramStack
from mintpy.utils import plot as pp, utils as ut
from mintpy import view, plot_network


mintpy_dir = ''

prj_dir = os.path.join(mintpy_dir, 'P1')

if mintpy_dir.endswith('/'):
    mintpy_dir = mintpy_dir[:-1]

if not os.path.isdir(prj_dir):
    os.mkdir(prj_dir)


# Alos_*.txt for ALOS, Sen_*.txt for Sentinel-1
prj_file = 'Sen_prj.txt'

os.chdir(prj_dir)

In [ ]:
config = """##-------------------------------- MintPy -----------------------------##
########## 1. Load Data (--load to exit after this step)
## load_data.py -H to check more details and example inputs.
mintpy.load.processor      = gamma
mintpy.load.unwFile        = {}/interferograms/*/diff_*rlks.unw
mintpy.load.corFile        = {}/interferograms/*/filt_*rlks.cor
mintpy.load.connCompFile   = None
mintpy.load.intFile        = None

mintpy.load.demFile        = {}/geometry/sim_*.rdc.dem
mintpy.load.lookupYFile    = {}/geometry/sim_*.UTM_TO_RDC
mintpy.load.lookupXFile    = {}/geometry/sim_*.UTM_TO_RDC
mintpy.load.incAngleFile   = None
mintpy.load.azAngleFile    = None
mintpy.load.shadowMaskFile = None
mintpy.load.waterMaskFile  = None
##---------subset (optional):
mintpy.subset.lalo         = auto    #[S:N,W:E / no], auto for no

########## 2. modify_network
mintpy.network.excludeIfgIndex = auto  #[1:5,25 / no], auto for no, list of ifg index (start from 0)
mintpy.network.keepMinSpanTree = yes  #[yes / no], auto for yes, keep interferograms in Min Span Tree network

########## 3. reference_point
mintpy.reference.lalo          = auto   #[31.8,130.8 / auto]

########## 4. correct_unwrap_error (optional)

########## 5. invert_network
mintpy.networkInversion.minTempCoh      = 0.5 #[0.0-1.0], auto for 0.7, min temporal coherence for mask

########## 6. correct_troposphere (optional and recommended)
mintpy.troposphericDelay.method         = height_correlation   #[pyaps / height_correlation / no], auto for pyaps

########## 7. deramp (optional)
mintpy.deramp                           = linear

########## 8. correct_topography (optional and recommended)

########## 9.1 residual_RMS (root mean squares for noise evaluation)

########## 9.2 reference_date

########## 10. velocity

########## 11.1 geocode (post-processing)

########## 11.2 google_earth (post-processing)

########## 11.3 hdfeos5 (post-processing)

########## 11.4 plot
""".format(mintpy_dir, mintpy_dir, mintpy_dir, mintpy_dir, mintpy_dir)

with open(prj_file, 'w+') as f:
    f.write(config)

## load_data

In [ ]:
!smallbaselineApp.py $prj_file --dostep load_data

## modify_network

In [ ]:
!smallbaselineApp.py $prj_file --dostep modify_network

In [ ]:
plot_network.main(['./inputs/ifgramStack.h5'])

## reference_point

In [ ]:
!smallbaselineApp.py $prj_file --dostep reference_point

In [ ]:
!info.py ./inputs/ifgramStack.h5 | egrep 'REF_'

## correct_unwrap_error

In [ ]:
!smallbaselineApp.py $prj_file --dostep correct_unwrap_error

## invert_network

In [ ]:
!smallbaselineApp.py $prj_file --dostep invert_network

In [ ]:
view.main('timeseries.h5 --wrap --wrap-range -5 5 -u cm --notitle --notick --noaxis'.split())

In [ ]:
view.main('temporalCoherence.h5 --notick --noaxis --noverbose'.split())
view.main('maskTempCoh.h5 --notick --noaxis --noverbose'.split())

## correct_troposphere

In [ ]:
!smallbaselineApp.py $prj_file --dostep correct_troposphere

In [ ]:
view.main('timeseries_tropHgt.h5 --wrap --wrap-range -5 5 -u cm --notitle --notick --noaxis'.split())

## deramp

In [ ]:
!smallbaselineApp.py $prj_file --dostep deramp

## correct_topography

In [ ]:
!smallbaselineApp.py $prj_file --dostep correct_topography

In [ ]:
view.main('timeseries_tropHgt_ramp_demErr.h5 --notitle --wrap --wrap-range -5 5 --notick --noaxis'.split())

## residual_RMS

In [ ]:
!smallbaselineApp.py $prj_file --dostep residual_RMS

In [ ]:
!head rms_timeseriesResidual_ramp.txt

## reference_date

In [ ]:
!cat reference_date.txt    #auto selected optimal reference date.

In [ ]:
## plot the RMS value
from mintpy.timeseries_rms import plot_rms_bar
txtContent = np.loadtxt('./rms_timeseriesResidual_ramp.txt', dtype=bytes).astype(str)
rms_list = [float(i) for i in txtContent[:, 1]]
date_list = [i for i in txtContent[:, 0]]
fig, ax = plt.subplots(figsize=[10, 4])
ax = plot_rms_bar(ax, date_list, rms_list)
plt.show()

In [ ]:
!smallbaselineApp.py $prj_file --dostep reference_date

## velocity

In [ ]:
!smallbaselineApp.py $prj_file --dostep velocity

In [ ]:
!info.py velocity.h5 --compact

In [ ]:
view.main('velocity.h5 --notick --noaxis --noverbose'.split())

## geocode

In [ ]:
!smallbaselineApp.py $prj_file --dostep geocode

In [ ]:
view.main('./geo/geo_velocity.h5 velocity --dem ./geo/geo_geometryRadar.h5 --figsize 12 12 --notick --noaxis'.split())

## google_earth

In [ ]:
!smallbaselineApp.py $prj_file --dostep google_earth

## save kmz

In [ ]:
!save_kmz.py ./geo/geo_velocity.h5 -u mm -v -60 60 -c jet -o ./geo/geo_velocity_mm

## plot_smallbaselineApp

In [ ]:
!${MINTPY_HOME}/mintpy/sh/plot_smallbaselineApp.sh